<a href="https://colab.research.google.com/github/benNthen/COMP723_DataMining/blob/main/TextClassification_drugTest_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00461/drugLib_raw.zip 

--2020-10-21 03:25:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00461/drugLib_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133354 (1.1M) [application/x-httpd-php]
Saving to: ‘drugLib_raw.zip’

drugLib_raw.zip     100%[===================>]   1.08M  2.81MB/s    in 0.4s    

2020-10-21 03:25:09 (2.81 MB/s) - ‘drugLib_raw.zip’ saved [1133354/1133354]



In [ ]:
!unzip drugLib_raw.zip          #unzip data 

Archive:  drugLib_raw.zip
  inflating: drugLibTest_raw.tsv     
  inflating: drugLibTrain_raw.tsv    


In [ ]:
import pandas as pd

#Import train data as x1 and test data as x2
x1 = pd.read_csv('drugLibTrain_raw.tsv', delimiter='\t') # train set
x2 = pd.read_csv('drugLibTest_raw.tsv',delimiter='\t') # test set


In [ ]:
import nltk
import numpy as np

from nltk.stem import WordNetLemmatizer
from sklearn import metrics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from nltk.tokenize import word_tokenize
nltk.download('corpus')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

lemmitizer = WordNetLemmatizer()

#lematize will simplify all the words in the corpus's feature column
def lemmword(corpus):
    L_corpus = []
    for i in corpus:
        if isinstance(i, str):
            tokens = word_tokenize(i)
            s = [lemmitizer.lemmatize(word) for word in tokens] # lematizes all tokens
            s = [word.lower() for word in s] # convert all letters into lowercases
            s = ' '.join(s) # reassembles tokens into one string
        else:
            s = ""
        L_corpus.append(s) # gets appended to final list
    return L_corpus

# function vector that converts strings from datasets into word vector, while filtering out stopwords
conv = CountVectorizer(stop_words=stopwords.words('english'))

# vectorizer used to convert review strings and returns a Term-Frequency Document Inverse Frequency(TF-DIF)
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words = stopwords.words('english'),
                             max_features=20000, max_df=0.60,
                             min_df=15, smooth_idf=True)

# function vector that converts strings from datasets into word vector, while filtering out stopwords
conv = CountVectorizer(stop_words=stopwords.words('english'))

# vectorizer used to convert review strings and returns a Term-Frequency Document Inverse Frequency(TF-DIF)
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words = stopwords.words('english'),
                             max_features=20000, max_df=0.60,
                             min_df=15, smooth_idf=True)

# columns features by their order in the array, used with iloc
#'Unnamed'(0), 'urlDrugName'(1), 'rating'(2), 'effectiveness'(3), 'sideEffects'(4),
# 'condition'(5), 'benefitsReview'(6), 'sideEffectsReview'(7), 'commentsReview(8)'

# extract column named 'rating' from train and test
rate_train = x1['rating'].values
rate_test = x2['rating'].values

# extract all contents of effectiveness by rows, convert to vector-array-list
effectiveness1 = conv.fit_transform(x1['effectiveness'].values.tolist()).toarray()
effectiveness2 = conv.transform(x2['effectiveness'].values.tolist()).toarray()

# extract all values column #4 - sideEffects on the train and test file
labels1 = x1.iloc[:, 4].values
labels2 = x2.iloc[:, 4].values

effRate = []
# effRate gives a '5-step rating' characteristic to the data by creating a key-value pair with 'effectiveness' attribute
# {'1': 'Ineffective', '2': 'Marginally Effective',
#  '3': 'Moderately Effective, '4': 'Considerably Effective',
#  '5': 'Highly Effective'}

# for train data
for effect in x1['effectiveness'].values:
    if effect == "Highly Effective":
        effRate.append([5])
    elif effect == "Moderately Effective":
        effRate.append([3])
    elif effect == "Considerably Effective":
        effRate.append([4])
    elif effect == "Marginally Effective":
        effRate.append([2])
    elif effect == "Ineffective":
        effRate.append([1])

ratingScore1 = x1[['rating']].values # use this variable to obtain organized test data set

# for test data
for effect in x2['effectiveness'].values:
    if effect == "Highly Effective":
        effRate.append([5])
    elif effect == "Moderately Effective":
        effRate.append([3])
    elif effect == "Considerably Effective":
        effRate.append([4])
    elif effect == "Marginally Effective":
        effRate.append([2])
    elif effect == "Ineffective":
        effRate.append([1])

ratingScore2 = x2[['rating']].values # use this variable to obtain organized train data set

# convert numpy array into a list
effectivenessRating1 = x1['effectiveness'].values.tolist()
effectivenessRating2 = x2['effectiveness'].values.tolist()

# extract and convert effectivenessRating to a vector
effRatingVec1 = conv.fit_transform(effectivenessRating1).toarray()
effRatingVec2 = conv.transform(effectivenessRating2).toarray()

# conversion from pandas to numpy to python list of the 'sideEffectsReview' feature column
sideEffComm1 = x1.iloc[:, 7].values.astype('str').tolist()
sideEffComm2 = x2.iloc[:, 7].values.astype('str').tolist()

# each row of the sideEffectsReviews are pre-processed through lemmword()
sideEffLem1 = lemmword(sideEffComm1)
sideEffLem2 = lemmword(sideEffComm2)

# To get a TF-IDF instead of a count vectoriser
sideEffLem1 = vectorizer.fit_transform(sideEffLem1).toarray()
sideEffLem2 = vectorizer.transform(sideEffLem2).toarray()

# numpy to python list of type unicode ('U')
commentsReview1 = x1.iloc[:, 8].values.astype('U').tolist()
commentsReview2 = x2.iloc[:, 8].values.astype('U').tolist()

# extract and convert effectivenessRating to a vector
commentsReviewVec1 = vectorizer.fit_transform(commentsReview1).toarray()
commentsReviewVec2 = vectorizer.transform(commentsReview2).toarray()

# conflate all reviews into one review set vector
train_row = np.concatenate((ratingScore1, effRatingVec1, commentsReviewVec1), axis=1)
test_row = np.concatenate((ratingScore2, effRatingVec2, commentsReviewVec2), axis=1)

# create label variables for column features with X for train, y for test
X = np.asarray(test_row)
y = labels2

# split train and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
classBay = GaussianNB()
NBmodel = classBay.fit(X_train, y_train)
NBpredict = NBmodel.predict(X_test)

print("Naives Bayes Accuracy:", metrics.accuracy_score(y_test, NBpredict))

Naives Bayes Accuracy: 0.2765273311897106


In [ ]:
# Neural Network Classifier
from sklearn import neural_network

ANclf = neural_network.MLPClassifier(max_iter=600, hidden_layer_sizes=(150, 150), early_stopping=False)
ANmodel = ANclf.fit(X_train, y_train)
ANpredict = ANmodel.predict(X_test)

# Neural Network Accuracy
print("Neural Network Accuracy:", metrics.accuracy_score(y_test, ANpredict))

Neural Network Accuracy: 0.37942122186495175


In [ ]:
# Decision Tree Classifier
from sklearn import tree
Tmodel = tree.DecisionTreeClassifier()
TClas= Tmodel.fit(X_train, y_train)
y_dec_tree = Tmodel.predict(X_test)

# Decision Tree Classifier Accuracy
print("Decision Tree Classifier Accuracy:", metrics.accuracy_score(y_test, y_dec_tree))

Decision Tree Classifier Accuracy: 0.3633440514469453


In [ ]:
#Random Forest Classifier
from sklearn import ensemble

RFclf = ensemble.RandomForestClassifier(n_estimators=600, max_leaf_nodes=400, max_depth=100)
RFClas = RFclf.fit(X=X_train, y=y_train)
y_pred_tree = RFclf.predict(X_test)

# Random Forest Classifier Accuracy
print("RandomForestClassifier Accuracy:", metrics.accuracy_score(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

RandomForestClassifier Accuracy: 0.42765273311897106
                               precision    recall  f1-score   support

Extremely Severe Side Effects       0.39      0.65      0.49        17
            Mild Side Effects       0.36      0.52      0.43        90
        Moderate Side Effects       0.42      0.43      0.42        77
              No Side Effects       0.57      0.43      0.49        84
          Severe Side Effects       0.55      0.14      0.22        43

                     accuracy                           0.43       311
                    macro avg       0.46      0.43      0.41       311
                 weighted avg       0.46      0.43      0.42       311



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNclf = KNeighborsClassifier(n_neighbors=20)
KNmod = KNclf.fit(X_train, y_train)
y_pred_knn = KNmod.predict(X_test)

# K-Nearest Neighbor Classifier Accuracy
print("K-Nearest Neighbor Accuracy:", metrics.accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbor Accuracy: 0.5466237942122186
                               precision    recall  f1-score   support

Extremely Severe Side Effects       0.48      0.82      0.61        17
            Mild Side Effects       0.51      0.64      0.57        90
        Moderate Side Effects       0.55      0.61      0.58        77
              No Side Effects       0.67      0.49      0.57        84
          Severe Side Effects       0.45      0.23      0.31        43

                     accuracy                           0.55       311
                    macro avg       0.53      0.56      0.53       311
                 weighted avg       0.55      0.55      0.54       311

